In [33]:
# This is practice for convoluted neural networks
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

In [40]:
# Step 1 - Preparing the data for the model
from datasets import load_dataset

ds = load_dataset("ylecun/mnist")

# Splitting data to proper sections
train_ds = ds['train']
val_ds = ds['test']

# Define a transform to downsize, convert to grayscale, and normalize images
transform = transforms.Compose([
    transforms.Resize((28, 28)),          # Downsize images to 28x28 pixels
    transforms.Grayscale(),               # Convert images to grayscale
    transforms.ToTensor(),                # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize with mean 0.5 and std 0.5
])

# Custom transform function to handle the dataset format
def custom_transform(img_dict):
    image = img_dict['image']
    label = img_dict['label']
    image = transforms.ToPILImage()(image)  # Convert to PIL image
    image = transform(image)  # Apply the transforms
    return {'image': image, 'label': label}

# Apply the custom transform to the dataset
train_ds = ds['train'].map(custom_transform, batched=False)
val_ds = ds['test'].map(custom_transform, batched=False)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=64, shuffle=False)

In [35]:
# Step 2 - Creating the Validator
def validate(model, train_loader, val_loader):
    # Loops through the training and validation sets
    for name, loader in [('train', train_loader),('val', val_loader)]:
        correct = 0
        total = 0

        # Disable gradient computation since we are only doing inference
        with torch.no_grad():

            # Loops through each image in the train + val sets
            for imgs, labels in loader:

                # Sends the images and labels to the appropriate device (GPU or CPU)
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
    
                # Passes the images through the model to get predicted values
                outputs = model(imgs)
    
                # Retrieves the index of Maximum probability for each vector of predictions
                _, predicted = torch.max(outputs, dim=1)
    
                # Updates the running total of samples and correct predictions
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        # Calculate and prints the accuracy for the current dataset
        print("Accuracy {}: {:.2f}".format(name , correct / total))            

In [36]:
# Step 3 - Creating the Residual Block in the Neural Network
class ResBlock(nn.Module):

    # Initializing the class
    def __init__(self, n_chans):
        # Initializing the parent class to ensure proper setup
        super(ResBlock, self).__init__()

        # Creating the convolution that will be applied to the input
        self.conv = nn.Conv2d(1,n_chans,kernel_size=2,padding=1,bias=False)

        # Batch normalization to normalize the activations
        self.batch_norm = nn.BatchNorm2d(num_features=n_chans)

        # Initializing the weights of the convolutional layer with Kaiming Normal initialization
        # This makes weights follow a normal distribution with a mean of 0,
        # and a standard deviation that depends on the number of input units
        torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')

        # Initializing batch normalization parameters
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    # Defining the forward pass of the models, which dictates how data moves through the network
    def forward(self, x):
        # Applies the convolutional layer to the input
        out = self.conv(x)

        # Applies the batch normalization layer to the input
        out = self.batch_norm(out)

        # Applies the ReLU activation function to introduce non-linearity
        out = torch.relu(out)

        # Adds the input to the output (residual connection)
        # This helps to mitigate the vanishing gradient problem and allows earlier gradients
        # to impact the output of the model
        return out + x

In [37]:
# Step 4 - Creating the residual model
class ResNet(nn.Module):
    # Initializing the variables in the class
    def __init__(self, n_chans1=28, n_blocks=10):
        
        # Initializing the parent class
        super().__init__()

        # Initializes the width of each layer
        self.n_chans1 = n_chans1

        # Creating the convolution to be applied to each layer
        self.conv = nn.Conv2d(1, n_chans1, kernel_size=1, padding=1, bias=False)

        # Creating the Residual Blocks that will be stacked for the input to flow through
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))

        # Creating the fully connected networks to tie it all together
        # Primarily to restrict data to feed into output (10, 0-9)
        self.fc1 = nn.Linear(8 * 8 * self.n_chans1, 28)
        self.fc2 = nn.Linear(28, 10)

    # Creating the forward pass for how the data goes through the network
    def forward(self, x):
        # Applies the Functional version of MaxPool to identify structures, ReLU activation to introduce non-linearity, and runs the convolution
        out = F.max_pool2d(torch.relu(self.conv(x)),2)

        # Sends the data through the ResBlocks
        out = self.resblocks(out)

        # Applies the max pool to identify further structures in the image
        out = F.max_pool2d(out, 2)

        # Flattens out the network from 2d to 1d as fully connected networks require 1d
        out = out.view(-1, 8 * 8 * self.n_chans1)

        # Starts sending the data through the finally activation function and fully connected networks
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [38]:
# Step 5 - Creating the training loop function
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            # Sends the images and labels to appropriate device (GPU or CPU)
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)

            # Runs the images through the model to get predicted values
            outputs = model(imgs)

            # Calculates the loss by comparing the predicted values to the actual labels
            loss = loss_fn(outputs, labels)

            # Zeros the gradients to prevent accumulation from previous iterations
            optimizer.zero_grad()

            # Backpropogates the loss to compute the gradient of loss with respect to model parameters
            optimizer.backward()

            # Updates the model's weights and biases based on the computed gradients
            optimizer.step()

            # Accumulates the training loss for monitoring purposes
            loss_train += loss.item

        # Prints the average training loss
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training Loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [41]:
# Step 6 - Calling the training loop to train + validate the model

# Assigns the model to the ResNet class, initializing it with the specified parameters
model = ResNet(n_chans1 = 28, n_blocks=10)

# Creates the optimizer with the model parameters and the specified learning rate
optimizer = optim.SGD(model.parameters(), lr=3e-3)

# Chooses + defines the loss function
# Specifically choosing the cross entropy loss to make values less that .5 probability have an exponential slope
# Suitable loss function for classification tasks
loss_fn = nn.CrossEntropyLoss()

# Call the training loop to train the model
training_loop(
    n_epochs=10,                 # Number of epochs for training
    optimizer = optimizer,       # Optimizer for updating the model parameters
    model = model,               # The model to be trained
    loss_fn = loss_fn,           # Loss function for calculating the loss
    train_loader = train_loader  # Data loader for the training data
)

TypeError: pic should be PIL Image or ndarray. Got <class 'dict'>

In [26]:
img = ds['train'][0]
ds.shape

{'train': (60000, 2), 'test': (10000, 2)}

In [19]:
img['image']

In [27]:
img.shape

AttributeError: 'dict' object has no attribute 'shape'